In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_kbo_rank(date=None):
    """
    KBO 공식 사이트의 일자별 팀 순위를 가져옵니다.
    date: 'YYYY-MM-DD' 형태. None이면 오늘 날짜.
    """
    url = "https://www.koreabaseball.com/record/teamrank/teamrankdaily.aspx"
    params = {}
    if date:
        params['gameDate'] = date  # 페이지에 따라 다를 수 있음; 개발자 도구 확인
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers, params=params)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")
    
    table = soup.select_one("table.tData")  # 실제 CSS 클래스는 확인 필요
    data = []
    if table:
        for row in table.select("tbody tr"):
            cols = [td.get_text(strip=True) for td in row.find_all("td")]
            if len(cols) >= 10:
                data.append({
                    '순위': cols[0],
                    '팀명': cols[1],
                    '경기': cols[2],
                    '승': cols[3],
                    '패': cols[4],
                    '무': cols[5],
                    '승률': cols[6],
                    '게임차': cols[7],
                    '최근10경기': cols[8],
                    '연속': cols[9]
                })
    return pd.DataFrame(data)

if __name__ == "__main__":
    df = fetch_kbo_rank("2025-06-15")
    print(df)

   순위   팀명  경기   승   패  무     승률  게임차  최근10경기  연속
0   1   한화  69  41  27  1  0.603    0  6승1무3패  4승
1   2   LG  69  40  27  2  0.597  0.5  4승1무5패  1패
2   3   롯데  70  37  30  3  0.552  3.5  5승0무5패  1패
3   4   KT  70  36  31  3  0.537  4.5  6승0무4패  2승
4   5   삼성  69  36  32  1  0.529    5  5승0무5패  2패
5   6  SSG  68  34  32  2  0.515    6  4승0무6패  1승
6   7  KIA  68  34  33  1  0.507  6.5  5승0무5패  2승
7   8   NC  66  29  33  4  0.468    9  4승1무5패  2패
8   9   두산  69  27  39  3  0.409   13  4승0무6패  2승
9  10   키움  72  20  50  2  0.286   22  4승1무5패  5패


In [25]:
df

,순위,팀명,경기,승,패,무,승률,게임차,최근10경기,연속
0,1,한화,69,41,27,1,0.603,0,6승1무3패,4승
1,2,LG,69,40,27,2,0.597,0.5,4승1무5패,1패
2,3,롯데,70,37,30,3,0.552,3.5,5승0무5패,1패
3,4,KT,70,36,31,3,0.537,4.5,6승0무4패,2승
4,5,삼성,69,36,32,1,0.529,5,5승0무5패,2패
5,6,SSG,68,34,32,2,0.515,6,4승0무6패,1승
6,7,KIA,68,34,33,1,0.507,6.5,5승0무5패,2승
7,8,NC,66,29,33,4,0.468,9,4승1무5패,2패
8,9,두산,69,27,39,3,0.409,13,4승0무6패,2승
9,10,키움,72,20,50,2,0.286,22,4승1무5패,5패


In [27]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx'

html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

# 테이블 본문 (tbody) 찾기
tbody = soup.find('tbody')

# 데이터 저장 리스트
data = []

# 각 선수별 tr 태그 순회
for tr in tbody.find_all('tr'):
    tds = tr.find_all('td')
    if len(tds) < 19:
        continue  # 예상 컬럼 수 미달 시 건너뜀
    
    # 순위부터 WHIP까지 19개 컬럼
    row = [td.text.strip() for td in tds[:19]]
    data.append(row)

# 컬럼명
columns = ['순위', '선수명', '팀명', 'ERA', 'G', 'W', 'L', 'SV', 'HLD', 'WPCT', 'IP',
           'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', 'WHIP']

df = pd.DataFrame(data, columns=columns)

print(df)


    순위   선수명   팀명   ERA   G  W  L SV HLD   WPCT      IP   H  HR  BB HBP   SO  \
0    1   앤더슨  SSG  2.09  14  5  3  0   0  0.625      82  60   4  22   4  119   
1    2    폰세   한화  2.16  15  9  0  0   0  1.000      96  61   6  22   3  129   
2    3  헤이수스   KT  2.43  13  5  4  0   0  0.556      74  63   7  23   1   82   
3    4    네일  KIA  2.57  15  5  2  0   0  0.714      91  77   3  22  11   82   
4    5   임찬규   LG  2.61  14  8  2  0   0  0.800  86 1/3  77   3  25   3   59   
5    6   송승기   LG  2.65  13  7  4  0   0  0.636  74 2/3  56   5  26   1   69   
6    7   원태인   삼성  2.68  12  5  2  0   0  0.714      74  68   6   8   2   52   
7    8  치리노스   LG  2.84  14  7  2  0   0  0.778  85 2/3  76   3  19   4   78   
8    9   고영표   KT  2.93  13  6  4  0   0  0.600      80  91   2  16   5   81   
9   10   오원석   KT  2.93  13  8  3  0   0  0.727  73 2/3  62   5  31   4   65   
10  11   잭로그   두산  2.98  14  4  6  0   0  0.400  84 2/3  63   2  20   9   78   
11  12   후라도   삼성  2.99  15  6  6  0   0

In [ ]:
# 투수기록
df

,순위,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,IP,H,HR,BB,HBP,SO,R,ER,WHIP
0,1,앤더슨,SSG,2.09,14,5,3,0,0,0.625,82,60,4,22,4,119,23,19,1.00
1,2,폰세,한화,2.16,15,9,0,0,0,1.000,96,61,6,22,3,129,24,23,0.86
2,3,헤이수스,KT,2.43,13,5,4,0,0,0.556,74,63,7,23,1,82,28,20,1.16
3,4,네일,KIA,2.57,15,5,2,0,0,0.714,91,77,3,22,11,82,28,26,1.09
4,5,임찬규,LG,2.61,14,8,2,0,0,0.800,86 1/3,77,3,25,3,59,26,25,1.18
5,6,송승기,LG,2.65,13,7,4,0,0,0.636,74 2/3,56,5,26,1,69,23,22,1.10
6,7,원태인,삼성,2.68,12,5,2,0,0,0.714,74,68,6,8,2,52,23,22,1.03
7,8,치리노스,LG,2.84,14,7,2,0,0,0.778,85 2/3,76,3,19,4,78,28,27,1.11
8,9,고영표,KT,2.93,13,6,4,0,0,0.600,80,91,2,16,5,81,32,26,1.34
9,10,오원석,KT,2.93,13,8,3,0,0,0.727,73 2/3,62,5,31,4,65,26,24,1.26


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

# 웹드라이버 실행
driver = webdriver.Chrome()

# KBO 기록 페이지 접속
url = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx"
driver.get(url)
time.sleep(3)

# 연도 선택 드롭다운 요소 찾기
select_element = Select(driver.find_element(By.ID, "cphContents_cphContents_cphContents_ddlSeason_ddlSeason"))
select_element.select_by_value("2002")  # 연도를 변경
time.sleep(2)

# 투수 기록 테이블 찾기
rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

# 데이터 추출
pitchers_data = []
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    data = [col.text for col in cols]
    data.insert(0, "2002")  # 연도 추가
    pitchers_data.append(data)

# 데이터 저장
df = pd.DataFrame(pitchers_data)
df.to_csv("kbo_pitchers_2002.csv", index=False)

driver.quit()
print("2002년 데이터 수집 완료!")

2002년 데이터 수집 완료!


In [14]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2002,1,엘비라,삼성,2.50,22,13,6,0,0,0.684,137,115,7,52,4,111,44,38,1.22
1,2002,2,송진우,한화,2.99,31,18,7,0,0,0.720,220,199,16,54,11,165,82,73,1.15
2,2002,3,임창용,삼성,3.08,36,17,6,2,0,0.739,204 1/3,188,24,34,24,160,82,70,1.09
3,2002,4,리오스,KIA,3.14,54,14,5,13,1,0.737,157 2/3,140,14,44,24,102,59,55,1.17
4,2002,5,이승호,SK,3.15,27,6,12,0,0,0.333,142 2/3,120,18,40,5,144,55,50,1.12
5,2002,6,채병용,SK,3.19,51,7,9,11,2,0.438,141,120,20,44,9,126,51,50,1.16
6,2002,7,키퍼,KIA,3.34,32,19,9,0,0,0.679,202 1/3,172,24,52,16,135,84,75,1.11
7,2002,8,박명환,두산,3.44,30,14,10,0,0,0.583,185 2/3,173,12,68,7,169,78,71,1.30
8,2002,9,손민한,롯데,3.67,23,4,9,0,0,0.308,142 1/3,144,19,30,4,82,68,58,1.22
9,2002,10,구자운,두산,3.79,32,8,8,1,0,0.500,133,134,6,58,7,95,64,56,1.44


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 웹드라이버 실행
driver = webdriver.Chrome()

# KBO 기록 페이지 접속
url = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx"
driver.get(url)
time.sleep(3)

# 크롤링할 연도 범위
start_year = 2002
end_year = 2025  # 최신 연도까지

# 데이터 저장을 위한 리스트
all_data = []

# 연도별 데이터 크롤링
for year in range(start_year, end_year + 1):
    try:
        # 연도 선택 드롭다운 요소 찾기
        select_element = Select(driver.find_element(By.ID, "cphContents_cphContents_cphContents_ddlSeason_ddlSeason"))
        select_element.select_by_value(str(year))
        time.sleep(2)  # 페이지 로딩 대기
        
        # 투수 기록 테이블의 모든 행
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

        # 데이터 추출
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            # 랭크 빼고 18개 컬럼만 추출 (선수명부터 WHIP까지)
            data = [col.text for col in cols[1:19]]  # 1번 인덱스부터 18개
            data.insert(0, year)  # 연도 추가
            all_data.append(data)

        print(f"{year}년 데이터 수집 완료")
    
    except Exception as e:
        print(f"{year}년 데이터 수집 중 오류 발생: {e}")

# 컬럼명: Rank 없이 연도 + 18개
column_names = ["Year", "선수명", "팀명", "ERA", "G", "W", "L", "SV", "HLD",
                "WPCT", "IP", "H", "HR", "BB", "HBP", "SO", "R", "ER", "WHIP"]

df = pd.DataFrame(all_data, columns=column_names)

# CSV 파일로 저장
df.to_csv("kbo_pitchers_(2002-2025).csv", index=False)

print("모든 연도 데이터 저장 완료!")

# WebDriver 종료
driver.quit()

# 연도별 데이터 조회 함수
def get_year_data(year):
    return df[df["Year"] == year]


2002년 데이터 수집 완료
2003년 데이터 수집 완료
2004년 데이터 수집 완료
2005년 데이터 수집 완료
2006년 데이터 수집 완료
2007년 데이터 수집 완료
2008년 데이터 수집 완료
2009년 데이터 수집 완료
2010년 데이터 수집 완료
2011년 데이터 수집 완료
2012년 데이터 수집 완료
2013년 데이터 수집 완료
2014년 데이터 수집 완료
2015년 데이터 수집 완료
2016년 데이터 수집 완료
2017년 데이터 수집 완료
2018년 데이터 수집 완료
2019년 데이터 수집 완료
2020년 데이터 수집 완료
2021년 데이터 수집 완료
2022년 데이터 수집 완료
2023년 데이터 수집 완료
2024년 데이터 수집 완료
2025년 데이터 수집 완료
모든 연도 데이터 저장 완료!


In [28]:
# 2010년 데이터 조회
year_to_check = 2010
filtered_df = get_year_data(year_to_check)
print(filtered_df)

     Year   선수명   팀명   ERA   G   W   L SV HLD   WPCT       IP    H  HR  BB  \
150  2010   류현진   한화  1.82  25  16   4  0   0  0.800  192 2/3  149  11  45   
151  2010   김광현   SK  2.37  31  17   7  0   0  0.708  193 2/3  153  13  84   
152  2010  카도쿠라   SK  3.22  30  14   7  0   0  0.667  153 2/3  146  14  52   
153  2010  히메네스   두산  3.32  27  14   5  0   0  0.737      152  139   7  62   
154  2010   서재응  KIA  3.34  24   9   7  0   0  0.563      140  134  10  29   
155  2010   장원삼   삼성  3.46  29  13   5  0   0  0.722      151  139  13  44   
156  2010   봉중근   LG  3.58  28  10   9  0   0  0.526  178 1/3  166  13  69   
157  2010  사도스키   롯데  3.87  27  10   8  0   0  0.556  169 2/3  153  17  54   
158  2010   김선우   두산  4.02  28  13   6  0   0  0.684  154 2/3  163  12  50   
159  2010   양현종  KIA  4.25  30  16   8  0   0  0.667  169 1/3  169   9  98   
160  2010   송승준   롯데  4.39  27  14   6  0   0  0.700      168  156  20  60   
161  2010   장원준   롯데  4.43  26  12   6  0   0  0.667  144 1/3  1

In [29]:
filtered_df

,Year,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,IP,H,HR,BB,HBP,SO,R,ER,WHIP
150,2010,류현진,한화,1.82,25,16,4,0,0,0.800,192 2/3,149,11,45,9,187,42,39,1.01
151,2010,김광현,SK,2.37,31,17,7,0,0,0.708,193 2/3,153,13,84,5,183,56,51,1.22
152,2010,카도쿠라,SK,3.22,30,14,7,0,0,0.667,153 2/3,146,14,52,6,143,65,55,1.29
153,2010,히메네스,두산,3.32,27,14,5,0,0,0.737,152,139,7,62,9,87,65,56,1.32
154,2010,서재응,KIA,3.34,24,9,7,0,0,0.563,140,134,10,29,11,68,53,52,1.16
155,2010,장원삼,삼성,3.46,29,13,5,0,0,0.722,151,139,13,44,5,115,62,58,1.21
156,2010,봉중근,LG,3.58,28,10,9,0,0,0.526,178 1/3,166,13,69,14,130,74,71,1.32
157,2010,사도스키,롯데,3.87,27,10,8,0,0,0.556,169 2/3,153,17,54,8,120,83,73,1.22
158,2010,김선우,두산,4.02,28,13,6,0,0,0.684,154 2/3,163,12,50,8,128,77,69,1.38
159,2010,양현종,KIA,4.25,30,16,8,0,0,0.667,169 1/3,169,9,98,6,145,84,80,1.58


In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 웹드라이버 실행
driver = webdriver.Chrome()

# KBO 기록 페이지 접속
url = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx"
driver.get(url)
time.sleep(3)

start_year = 1982
end_year = 2001

all_data = []

for year in range(start_year, end_year + 1):
    try:
        select_element = Select(driver.find_element(By.ID, "cphContents_cphContents_cphContents_ddlSeason_ddlSeason"))
        select_element.select_by_value(str(year))
        time.sleep(2)
        
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) < 21:
                continue  # 컬럼 개수 미달 시 건너뜀
            
            # 순위는 첫 번째 td이므로 제외하고 나머지 20개 텍스트 추출
            data = [col.text for col in cols[1:21]]
            data.insert(0, year)  # 연도 추가
            all_data.append(data)
        
        print(f"{year}년 데이터 수집 완료")
    except Exception as e:
        print(f"{year}년 데이터 수집 중 오류 발생: {e}")

# 컬럼명 (Year 포함 21개)
column_names = ["Year", "선수명", "팀명", "ERA", "G", "CG", "SHO", "W", "L", "SV", "HLD", "WPCT", "TBF",
                "IP", "H", "HR", "BB", "HBP", "SO", "R", "ER"]

df = pd.DataFrame(all_data, columns=column_names)

df.to_csv("kbo_pitchers_(1982-2001).csv", index=False)

print("1982-2001년 데이터 저장 완료!")

driver.quit()


1982년 데이터 수집 완료
1983년 데이터 수집 완료
1984년 데이터 수집 완료
1985년 데이터 수집 완료
1986년 데이터 수집 완료
1987년 데이터 수집 완료
1988년 데이터 수집 완료
1989년 데이터 수집 완료
1990년 데이터 수집 완료
1991년 데이터 수집 완료
1992년 데이터 수집 완료
1993년 데이터 수집 완료
1994년 데이터 수집 완료
1995년 데이터 수집 완료
1996년 데이터 수집 완료
1997년 데이터 수집 완료
1998년 데이터 수집 완료
1999년 데이터 수집 완료
2000년 데이터 수집 완료
2001년 데이터 수집 완료
1982-2001년 데이터 저장 완료!


In [7]:
# 연도별 데이터 조회 함수
def get_year_data(year):
    return df[df["Year"] == year]

In [8]:
# 2010년 데이터 조회
year_to_check = 1990
filtered_df = get_year_data(year_to_check)
print(filtered_df)

     Year  선수명   팀명   ERA   G  CG SHO   W   L  SV  ...   WPCT  TBF       IP  \
209  1990  선동열   해태  1.13  35   8   6  22   6   4  ...  0.786  724  190 1/3   
210  1990  송진우  빙그레  1.82  50   0   0  11   7  27  ...  0.611  503  128 1/3   
211  1990  김용수   LG  2.04  33   2   0  12   5   5  ...  0.706  594  149 2/3   
212  1990  김태원   LG  2.51  33   6   3  18   5   1  ...  0.783  786  193 1/3   
213  1990  한용덕  빙그레  2.53  38   3   1  13   9   3  ...  0.591  706      171   
214  1990  박정현  태평양  2.63  35   9   2  13   7   7  ...  0.650  782  191 2/3   
215  1990  정삼흠   LG  2.78  55   1   0   8   9  23  ...  0.471  684  168 1/3   
216  1990  김상엽   삼성  2.81  44   3   0  12   6  18  ...  0.667  661  160 1/3   
217  1990  박동희   롯데  3.04  31   3   1  10   7   7  ...  0.588  629      145   
218  1990  이강철   해태  3.14  44  10   1  16  10   5  ...  0.615  894  220 2/3   
219  1990  최창호  태평양  3.20  40   5   1   9   9   5  ...  0.500  824  196 2/3   
220  1990  양상문  태평양  3.22  34   6   4  11   9   1  .

In [9]:
filtered_df

,Year,선수명,팀명,ERA,G,CG,SHO,W,L,SV,...,WPCT,TBF,IP,H,HR,BB,HBP,SO,R,ER
209,1990,선동열,해태,1.13,35,8,6,22,6,4,...,0.786,724,190 1/3,121,1,50,8,189,35,24
210,1990,송진우,빙그레,1.82,50,0,0,11,7,27,...,0.611,503,128 1/3,80,2,54,0,102,29,26
211,1990,김용수,LG,2.04,33,2,0,12,5,5,...,0.706,594,149 2/3,122,4,38,3,119,40,34
212,1990,김태원,LG,2.51,33,6,3,18,5,1,...,0.783,786,193 1/3,152,8,61,7,127,65,54
213,1990,한용덕,빙그레,2.53,38,3,1,13,9,3,...,0.591,706,171,147,10,58,7,100,59,48
214,1990,박정현,태평양,2.63,35,9,2,13,7,7,...,0.650,782,191 2/3,177,7,55,8,66,66,56
215,1990,정삼흠,LG,2.78,55,1,0,8,9,23,...,0.471,684,168 1/3,133,4,57,9,86,58,52
216,1990,김상엽,삼성,2.81,44,3,0,12,6,18,...,0.667,661,160 1/3,114,12,79,11,110,60,50
217,1990,박동희,롯데,3.04,31,3,1,10,7,7,...,0.588,629,145,101,3,114,3,146,55,49
218,1990,이강철,해태,3.14,44,10,1,16,10,5,...,0.615,894,220 2/3,163,24,84,9,165,84,77


In [57]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 크롬 드라이버 실행
driver = webdriver.Chrome()

# KBO 일정 페이지 접속
url = "https://www.koreabaseball.com/Schedule/Schedule.aspx"
driver.get(url)
time.sleep(3)

start_year = 2025
months = [f"{m:02}" for m in range(1, 13)]

all_data = []

for year in range(start_year, start_year + 1):
    for month in months:
        try:
            Select(driver.find_element(By.ID, "ddlYear")).select_by_value(str(year))
            Select(driver.find_element(By.ID, "ddlMonth")).select_by_value(month)
            time.sleep(2)

            rows = driver.find_elements(By.CSS_SELECTOR, "#tblScheduleList tbody tr")
            current_date = None

            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if not cols or len(cols) < 8:
                    continue

                if len(cols) == 9:
                    current_date = cols[0].text.strip().split('(')[0].strip()
                    time_ = cols[1].text.strip()
                    game = cols[2].text.strip()
                    tv = cols[5].get_attribute("textContent").strip()
                    radio = cols[6].get_attribute("textContent").strip()
                    stadium = cols[7].get_attribute("textContent").strip()
                    note = cols[8].get_attribute("textContent").strip()
                else:
                    time_ = cols[0].text.strip()
                    game = cols[1].text.strip()
                    tv = cols[4].get_attribute("textContent").strip()
                    radio = cols[5].get_attribute("textContent").strip()
                    stadium = cols[6].get_attribute("textContent").strip()
                    note = cols[7].get_attribute("textContent").strip()

                all_data.append([
                    year, current_date, time_, game, tv, radio, stadium, note
                ])

            print(f"{year}년 {month}월 완료")
        except Exception as e:
            print(f"❌ {year}년 {month}월 오류 발생:", e)

# 데이터프레임 생성
columns = ["연도", "날짜", "시간", "경기", "TV", "라디오", "구장", "비고"]
df = pd.DataFrame(all_data, columns=columns)

# CSV 저장
df.to_csv("kbo_schedule_2025_full.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 저장 완료!")

driver.quit()

2025년 01월 완료
2025년 02월 완료
2025년 03월 완료
2025년 04월 완료
2025년 05월 완료
2025년 06월 완료
2025년 07월 완료
2025년 08월 완료
2025년 09월 완료
2025년 10월 완료
2025년 11월 완료
2025년 12월 완료
✅ 전체 저장 완료!


In [58]:
# 날짜 문자열 형식이 '07.25'처럼 월.일 형태임을 고려
target_date = "05.25"
target_year = 2025

# 해당 날짜에 해당하는 경기만 필터링
selected_games = df[(df["연도"] == target_year) & (df["날짜"] == target_date)]

# 결과 출력
print(selected_games)

       연도     날짜     시간         경기          TV 라디오     구장 비고
287  2025  05.25  14:00   NC3vs5두산        KN-T         잠실  -
288  2025  05.25  14:00  LG9vs3SSG       SPO-T         문학  -
289  2025  05.25  14:00  KIA2vs3삼성        SS-T         대구  -
290  2025  05.25  14:00   KT2vs0키움  SPO-2TMS-T         고척  -
291  2025  05.25  14:00   롯데7vs8한화         M-T      대전(신)  -


In [59]:
selected_games

,연도,날짜,시간,경기,TV,라디오,구장,비고
287,2025,05.25,14:00,NC3vs5두산,KN-T,,잠실,-
288,2025,05.25,14:00,LG9vs3SSG,SPO-T,,문학,-
289,2025,05.25,14:00,KIA2vs3삼성,SS-T,,대구,-
290,2025,05.25,14:00,KT2vs0키움,SPO-2TMS-T,,고척,-
291,2025,05.25,14:00,롯데7vs8한화,M-T,,대전(신),-


In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 크롬 드라이버 실행
driver = webdriver.Chrome()

# KBO 일정 페이지 접속
url = "https://www.koreabaseball.com/Schedule/Schedule.aspx"
driver.get(url)
time.sleep(3)

start_year = 2001
end_year = 2025
months = [f"{m:02}" for m in range(1, 13)]

all_data = []

for year in range(start_year, end_year + 1):  # ✅ 범위 수정
    for month in months:
        try:
            Select(driver.find_element(By.ID, "ddlYear")).select_by_value(str(year))
            Select(driver.find_element(By.ID, "ddlMonth")).select_by_value(month)
            time.sleep(2)

            rows = driver.find_elements(By.CSS_SELECTOR, "#tblScheduleList tbody tr")
            current_date = None

            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if not cols or len(cols) < 8:
                    continue

                if len(cols) == 9:
                    current_date = cols[0].text.strip().split('(')[0].strip()
                    time_ = cols[1].text.strip()
                    game = cols[2].text.strip()
                    tv = cols[5].get_attribute("textContent").strip()
                    radio = cols[6].get_attribute("textContent").strip()
                    stadium = cols[7].get_attribute("textContent").strip()
                    note = cols[8].get_attribute("textContent").strip()
                else:
                    time_ = cols[0].text.strip()
                    game = cols[1].text.strip()
                    tv = cols[4].get_attribute("textContent").strip()
                    radio = cols[5].get_attribute("textContent").strip()
                    stadium = cols[6].get_attribute("textContent").strip()
                    note = cols[7].get_attribute("textContent").strip()

                all_data.append([year, current_date, time_, game, tv, radio, stadium, note])

            print(f"✅ {year}년 {month}월 완료")

        except Exception as e:
            print(f"❌ {year}년 {month}월 오류 발생:", e)

# 데이터프레임 생성
columns = ["연도", "날짜", "시간", "경기", "TV", "라디오", "구장", "비고"]
df = pd.DataFrame(all_data, columns=columns)

# CSV 저장
df.to_csv("kbo_schedule_2001_2025.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 저장 완료!")

driver.quit()


✅ 2001년 01월 완료
✅ 2001년 02월 완료
✅ 2001년 03월 완료
✅ 2001년 04월 완료
✅ 2001년 05월 완료
✅ 2001년 06월 완료
✅ 2001년 07월 완료
✅ 2001년 08월 완료
✅ 2001년 09월 완료
✅ 2001년 10월 완료
✅ 2001년 11월 완료
✅ 2001년 12월 완료
✅ 2002년 01월 완료
✅ 2002년 02월 완료
✅ 2002년 03월 완료
✅ 2002년 04월 완료
✅ 2002년 05월 완료
✅ 2002년 06월 완료
✅ 2002년 07월 완료
✅ 2002년 08월 완료
✅ 2002년 09월 완료
✅ 2002년 10월 완료
✅ 2002년 11월 완료
✅ 2002년 12월 완료
✅ 2003년 01월 완료
✅ 2003년 02월 완료
✅ 2003년 03월 완료
✅ 2003년 04월 완료
✅ 2003년 05월 완료
✅ 2003년 06월 완료
✅ 2003년 07월 완료
❌ 2003년 08월 오류 발생: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff6a69acda5+78885]
	GetHandleVerifier [0x0x7ff6a69ace00+78976]
	(No symbol) [0x0x7ff6a6769bca]
	(No symbol) [0x0x7ff6a67559b5]
	(No symbol) [0x0x7ff6a677a9ca]
	(No symbol) [0x0x7ff6a67f05e5]
	(No symbol) [0x0x7ff6a6810b42]
	(No symbol) [0x0x7ff6a67e8963]
	(No symbol) [0x0x7ff6a67b16b1]
	(No symbol) [0x0x7ff

In [62]:
# 날짜 문자열 형식이 '07.25'처럼 월.일 형태임을 고려
target_date = "05.25"
target_year = 2010

# 해당 날짜에 해당하는 경기만 필터링
selected_games = df[(df["연도"] == target_year) & (df["날짜"] == target_date)]

# 결과 출력
print(selected_games)

        연도     날짜     시간         경기         TV      라디오  구장 비고
4876  2010  05.25  18:30  KIA4vs5LG  KN-TSPO-T           잠실  -
4877  2010  05.25  18:30  두산7vs10롯데               KNN-R  사직  -
4878  2010  05.25  18:30  SK1vs14삼성             T-RDM-R  대구  -
4879  2010  05.25  18:30   넥센0vs2한화  MS-TD-CMB     TJ-R  대전  -


In [63]:
selected_games

,연도,날짜,시간,경기,TV,라디오,구장,비고
4876,2010,05.25,18:30,KIA4vs5LG,KN-TSPO-T,,잠실,-
4877,2010,05.25,18:30,두산7vs10롯데,,KNN-R,사직,-
4878,2010,05.25,18:30,SK1vs14삼성,,T-RDM-R,대구,-
4879,2010,05.25,18:30,넥센0vs2한화,MS-TD-CMB,TJ-R,대전,-


In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# 팀 코드 및 이름 매핑
teams = {
    "HT": "KIA", "SS": "삼성", "LG": "LG", "OB": "두산",
    "KT": "KT", "SK": "SSG", "LT": "롯데", "HH": "한화",
    "NC": "NC", "WO": "키움"
}

# 크롬 드라이버 실행
driver = webdriver.Chrome()
url = "https://www.koreabaseball.com/Schedule/Schedule.aspx"
driver.get(url)
time.sleep(3)

year = 2025
months = [f"{m:02}" for m in range(3, 13)]  # 3~12월

for team_code, team_name in teams.items():
    print(f"\n===== {team_name} 크롤링 시작 =====")
    team_data = []

    for month in months:
        try:
            # 연도와 월 선택
            Select(driver.find_element(By.ID, "ddlYear")).select_by_value(str(year))
            Select(driver.find_element(By.ID, "ddlMonth")).select_by_value(month)
            time.sleep(1)

            # 팀 탭 클릭 (ul > li[attr-value=팀코드])
            team_tab = driver.find_element(By.CSS_SELECTOR, f'ul.tab-schedule li[attr-value="{team_code}"]')
            driver.execute_script("arguments[0].click();", team_tab)
            time.sleep(2)

            # 경기 일정 테이블 대기
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#tblScheduleList tbody tr"))
            )

            rows = driver.find_elements(By.CSS_SELECTOR, "#tblScheduleList tbody tr")
            current_date = None

            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if not cols or len(cols) < 8:
                    continue

                if len(cols) == 9:
                    current_date = cols[0].text.strip().split('(')[0].strip()
                    time_ = cols[1].text.strip()
                    game = cols[2].text.strip()
                    tv = cols[5].text.strip()
                    radio = cols[6].text.strip()
                    stadium = cols[7].text.strip()
                    note = cols[8].text.strip()
                else:
                    time_ = cols[0].text.strip()
                    game = cols[1].text.strip()
                    tv = cols[4].text.strip()
                    radio = cols[5].text.strip()
                    stadium = cols[6].text.strip()
                    note = cols[7].text.strip()

                team_data.append([year, month, current_date, time_, game, tv, radio, stadium, note])

            print(f"✅ {team_name} - {year}년 {month}월 완료")

        except TimeoutException:
            print(f"⚠️ {team_name} - {year}년 {month}월: 경기 테이블 로딩 실패")
        except Exception as e:
            print(f"❌ {team_name} - {year}년 {month}월 오류:", e)

    # 저장
    if team_data:
        df = pd.DataFrame(team_data, columns=["연도", "월", "날짜", "시간", "경기", "TV", "라디오", "구장", "비고"])
        filename = f"kbo_schedule_2025_{team_name}.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"📁 {filename} 저장 완료!")

# 브라우저 종료
driver.quit()
print("\n🎉 모든 팀 일정 저장 완료!")


===== KIA 크롤링 시작 =====
✅ KIA - 2025년 03월 완료
✅ KIA - 2025년 04월 완료
✅ KIA - 2025년 05월 완료
✅ KIA - 2025년 06월 완료
✅ KIA - 2025년 07월 완료
✅ KIA - 2025년 08월 완료
✅ KIA - 2025년 09월 완료
✅ KIA - 2025년 10월 완료
✅ KIA - 2025년 11월 완료
✅ KIA - 2025년 12월 완료
📁 kbo_schedule_2025_KIA.csv 저장 완료!

===== 삼성 크롤링 시작 =====
✅ 삼성 - 2025년 03월 완료
✅ 삼성 - 2025년 04월 완료
✅ 삼성 - 2025년 05월 완료
✅ 삼성 - 2025년 06월 완료
✅ 삼성 - 2025년 07월 완료
✅ 삼성 - 2025년 08월 완료
✅ 삼성 - 2025년 09월 완료
✅ 삼성 - 2025년 10월 완료
✅ 삼성 - 2025년 11월 완료
✅ 삼성 - 2025년 12월 완료
📁 kbo_schedule_2025_삼성.csv 저장 완료!

===== LG 크롤링 시작 =====
✅ LG - 2025년 03월 완료
✅ LG - 2025년 04월 완료
✅ LG - 2025년 05월 완료
✅ LG - 2025년 06월 완료
✅ LG - 2025년 07월 완료
✅ LG - 2025년 08월 완료
✅ LG - 2025년 09월 완료
✅ LG - 2025년 10월 완료
✅ LG - 2025년 11월 완료
✅ LG - 2025년 12월 완료
📁 kbo_schedule_2025_LG.csv 저장 완료!

===== 두산 크롤링 시작 =====
✅ 두산 - 2025년 03월 완료
✅ 두산 - 2025년 04월 완료
✅ 두산 - 2025년 05월 완료
✅ 두산 - 2025년 06월 완료
✅ 두산 - 2025년 07월 완료
✅ 두산 - 2025년 08월 완료
✅ 두산 - 2025년 09월 완료
✅ 두산 - 2025년 10월 완료
✅ 두산 - 2025년 11월 완료
✅ 두산 - 2025년 1

In [24]:
import pandas as pd

df = pd.read_csv("kbo_schedule_2025_한화.csv")
df.fillna("", inplace=True)

# 3월 경기만 추출
march_games = df[df["월"] == 3]
march_games

C:\Users\Admin\AppData\Local\Temp\ipykernel_13340\411241874.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


,연도,월,날짜,시간,경기,TV,라디오,구장,비고
0,2025,3,3.22,14:00,한화4vs3KT,S-T,,수원,-
1,2025,3,3.23,14:00,한화4vs5KT,MS-T,,수원,-
2,2025,3,3.25,18:30,한화0vs5LG,SS-T,,잠실,-
3,2025,3,3.26,18:30,한화0vs4LG,SS-T,,잠실,-
4,2025,3,3.27,18:30,한화1vs2LG,SS-T,,잠실,-
5,2025,3,3.28,18:30,KIA2vs7한화,SPO-2T,,대전(신),-
6,2025,3,3.29,14:00,KIA4vs5한화,K-2T,,대전(신),-
7,2025,3,3.30,14:00,KIA5vs3한화,M-T,,대전(신),-


In [9]:
import pandas as pd

# 절대 경로로 파일 지정
file_path = "D:/hhm/code/baseball/team5-baseball-project/frontend/baseball_pj/public/oracle_game_data_result2.csv"

# CSV 파일 불러오기
df = pd.read_csv(file_path)

# 결측값 처리
df.fillna("", inplace=True)

# 날짜와 시간 분리
df[['날짜', '시간']] = df['GAME_DATE'].str.split(' ', expand=True)

# 연도 추출
df['연도'] = df['날짜'].str[:4]

# 열 이름 변경 및 순서 지정
df_renamed = df.rename(columns={
    'HOME_TEAM_NAME': '팀1',
    'AWAY_TEAM_NAME': '팀2',
    'HOME_SCORE': '점수1',
    'AWAY_SCORE': '점수2'
})

# 최종 정리된 결과 DataFrame
df_final = df_renamed[['연도', '날짜', '시간', '팀1', '점수1', '점수2', '팀2']]

# 결과 확인
print(df_final.head())

# 원하면 CSV로 저장
# df_final.to_csv("정제된_경기결과.csv", index=False, encoding="utf-8-sig")

     연도          날짜        시간  팀1   점수1  점수2   팀2
0  2001  2001-04-05  14:00:00  SK  11.0  6.0   LG
1  2001  2001-04-05  14:00:00  두산   6.0  5.0  KIA
2  2001  2001-04-05  14:00:00  현대   1.0  5.0   롯데
3  2001  2001-04-05  14:10:00  삼성   4.0  3.0   한화
4  2001  2001-04-06  18:30:00  삼성  12.0  3.0   한화


C:\Users\Admin\AppData\Local\Temp\ipykernel_28948\1202273155.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


In [11]:
import pandas as pd

file_path = "D:/hhm/code/baseball/team5-baseball-project/frontend/baseball_pj/public/oracle_game_data_result2.csv"
df = pd.read_csv(file_path)

# 문자열 열에만 빈 문자열 채움 (경고 해결)
df.loc[:, df.dtypes == 'object'] = df.loc[:, df.dtypes == 'object'].fillna("")

# 날짜, 시간, 연도 분리
df[['날짜', '시간']] = df['GAME_DATE'].str.split(' ', expand=True)
df['연도'] = df['날짜'].str[:4]

# 열 이름 재정렬 및 변경
df_renamed = df.rename(columns={
    'HOME_TEAM_NAME': '팀1',
    'AWAY_TEAM_NAME': '팀2',
    'HOME_SCORE': '점수1',
    'AWAY_SCORE': '점수2'
})
df_final = df_renamed[['연도', '날짜', '시간', '팀1', '점수1', '점수2', '팀2']]

# CSV 저장
df_final.to_csv("정제된_경기결과.csv", index=False, encoding="utf-8-sig")

In [12]:
import os
print("📁 현재 저장된 위치:", os.getcwd())

📁 현재 저장된 위치: d:\hhm\code\Note_s1\baseball
